In [1]:
# CLIENTE SOCKET PARA RECIBIR DATOS
# (SERVIDOR SOCKET EN Servidor.ipynb o PROYECTO.ipynb)

from socket import *
import pickle
import pandas as pd

# Nombre del servidor y puerto
serverName = '127.0.0.1' # Localhost
serverPort = 12000

# Crear socket TCP
clientSocket = socket(AF_INET, SOCK_STREAM)

try:
    # Conectar al servidor
    clientSocket.connect((serverName, serverPort))
    
    # Enviar frase/mensaje 
    decision = ""
    while decision not in ["JORNADA", "PARTIDO"]:
        decision = input("¿Deseas solicitar las predicciones de una jornada o de un solo partido? (JORNADA/PARTIDO): ").strip().upper()
        if decision not in ["JORNADA", "PARTIDO"]:
            print("Opción no válida. Por favor, escribe 'JORNADA' o 'PARTIDO'.")

    if decision == "PARTIDO":
        while True:
            division = input("Escribe la división (PRIMERA o SEGUNDA): ").strip().upper()
            if division in ["PRIMERA", "SEGUNDA"]:
                break
            else:
                print("División no válida. Por favor, escribe 'PRIMERA' o 'SEGUNDA'.")
        
        equipo_local = input("Escribe el nombre del equipo local: ").strip()
        equipo_visitante = input("Escribe el nombre del equipo visitante: ").strip()
        if division is not None and equipo_local is not None and equipo_visitante is not None:
            message = f"DAME_PREDICCIONES_PARTIDO|{division}|{equipo_local}|{equipo_visitante}"
    
    if decision == "JORNADA":
        while True:
            message = input("Escribe el comando de la división que deseas (DAME_PREDICCIONES_PRIMERA o DAME_PREDICCIONES_SEGUNDA): ").strip().upper()
            if message in ["DAME_PREDICCIONES_PRIMERA", "DAME_PREDICCIONES_SEGUNDA"]:
                break
            else:
                print("Comando no válido. Por favor, escribe 'DAME_PREDICCIONES_PRIMERA' o 'DAME_PREDICCIONES_SEGUNDA'.")

    clientSocket.send(message.encode())
    
    # Recibir respuesta (modifiedMessage en los apuntes)
    # NOTA: Como el DataFrame o el mensaje recibido es grande, necesitamos recibir en bucle
    # hasta que no lleguen más datos, a diferencia de recibir solo 1024 bytes una vez.
    if decision == "PARTIDO":
        # Recibimos bytes y usamos pickle para convertirlos a diccionario
        datos_bytes = clientSocket.recv(4096)
        if datos_bytes:
            datos_recibidos = pickle.loads(datos_bytes) # Convertimos bytes -> Diccionario
        else:
            datos_recibidos = None
    
    if decision == "JORNADA":
        datos_recibidos = b""
        while True:
            paquete = clientSocket.recv(4096) # Leemos paquetes de 4096 bytes
            if not paquete: 
                break
            datos_recibidos += paquete

    print('From Server: Datos recibidos completamente.')
    
    # Procesar y mostrar el DataFrame (Reconstruir el objeto)
    if datos_recibidos and decision == "PARTIDO":
        print(f"INFORME {equipo_local} vs {equipo_visitante}:")
        
        for seccion, contenido in datos_recibidos.items():
                print(f"\n[{seccion}]")
                if isinstance(contenido, dict):
                    for k, v in contenido.items():
                        print(f"  • {k}: {v}")
                else:
                    print(f"  {contenido}")

    if datos_recibidos and decision == "JORNADA":
        df_recibido = pickle.loads(datos_recibidos)
        
        print("TABLA DE PREDICCIONES RECIBIDA VÍA SOCKET")

        # Ajustes de visualización de pandas
        pd.set_option('display.max_colwidth', None)
        pd.set_option('display.width', 1000)
        
        print(df_recibido)

except Exception as e:
    print(f"Error de conexión: {e}")

# Cerrar el socket (Pág 22)
clientSocket.close()

From Server: Datos recibidos completamente.
INFORME Real Betis vs Valencia CF:

[1. CABECERA DEL PARTIDO]
  Real Betis (Pos: 6º) vs Valencia CF (Pos: 14º)

[2. PREDICCIÓN PRINCIPAL]
  • Resultado Esperado: Gana Real Betis
  • Marcador Estimado: 2 - 1
  • Goles Esperados (xG): 1.52 - 0.9
  • Confianza Local: Real Betis: 59.8%
  • Confianza Empate: 5%
  • Confianza Visitante: Valencia CF: 35.2%

[3. MOMENTO DE FORMA (ÚLTIMOS 5)]
  • Racha Real Betis: G-P-E-G-P (7 puntos de 15)
  • Últimos Rivales Real Betis: vs Getafe CF (21/12) | vs Real Madrid (04/01) | vs Real Oviedo (10/01) | vs Villarreal CF (17/01) | vs Deportivo Alavés (25/01)
  • Racha Valencia CF: E-P-E-G-G (8 puntos de 15)
  • Últimos Rivales Valencia CF: vs RCD Mallorca (19/12) | vs Celta de Vigo (03/01) | vs Elche CF (10/01) | vs Getafe CF (18/01) | vs RCD Espanyol (24/01)
  • Tendencia: Valencia CF llega mejor

[4. DATOS DE LA TEMPORADA]
  • Puntos en Liga: 32 vs 23
  • Media Goles A Favor: 1.62 vs 1.05
  • Media Goles En Co